In [17]:
import pandas as pd
import matplotlib as plt
import scipy.stats as stats
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.dummy import DummyClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.metrics import plot_confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.tree import DecisionTreeClassifier 
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier

In [2]:
df = pd.read_csv('data/weatherAUS.csv')

df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [ ]:
df.info()

In [ ]:
df.describe()

## Dealing with Missing Values

In [ ]:
#drop date, evaporation, sunshine(outliers),

In [3]:
df["Date"] = pd.to_datetime(df["Date"])   #Converting into Date
df["Month"] = df["Date"].dt.month         #abstracting month in diffrent column  
df.drop(columns='Date', inplace=True)
df.drop(columns='Evaporation', inplace=True)
df.drop(columns='Sunshine', inplace=True)
df.dropna(subset=['RainTomorrow'], inplace = True)

In [ ]:
df.isna().sum().sum()

In [ ]:
df.MinTemp.value_counts()
#either impute mean/median or drop nulls

In [ ]:
df.MaxTemp.value_counts()
# Same with MinTemp

In [ ]:
df.Rainfall.value_counts()
#

In [ ]:
df.RainToday.value_counts()
# For both rain today and rain tommorow we can drop missing values
# they account for less than 3 percent

In [ ]:
df.Cloud3pm.value_counts()
# 9 is a typo, we can swap that to 8
# 0 means clear skies and we can't really input that data without skewing

In [ ]:
df.Cloud9am.value_counts()
#swap 9 to 8, probably use median or mean to input values, edit: nah that shit
#aint gonna work chief

dataa

In [ ]:
#X = df.loc['Cloud9am'].values
  
#from sklearn.impute import SimpleImputer
#imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
#imputer = imputer.fit(X)
  
#X = imputer.transform(X)
#print(X)

In [4]:
df2 = df.copy()

In [5]:
features = ['MinTemp', 'MaxTemp', 'Rainfall', 'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am',
            'Humidity3pm', 'Pressure9am', 'Pressure3pm', 'Temp9am', 'Temp3pm']
for feature in features:
    qupper = df2[feature].quantile(0.99)
    df2 = df2.loc[(df2[feature].isna()) | (df2[feature] < qupper)]

In [6]:
X = df2.drop('RainTomorrow', axis=1)
y = df2['RainTomorrow']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [10]:
cols_simp = ['WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday']
simputer = SimpleImputer(strategy = 'most_frequent')
simputed = simputer.fit_transform(X_train[cols_simp])
simptest = simputer.transform(X_test[cols_simp])
df_simp_tranformed = pd.DataFrame(simputed, index=X_train.index, columns=cols_simp)
df_simp_tranformed

,WindGustDir,WindDir9am,WindDir3pm,RainToday
2867,NNW,SE,NE,No
110045,W,SW,SE,No
55289,N,NE,NNE,No
39713,W,WNW,WNW,No
114443,S,ESE,SSE,No
...,...,...,...,...
139707,WNW,WSW,WNW,No
121459,SE,SE,SE,No
940,E,NW,ESE,No
18212,S,SW,SSE,No


In [11]:
X_train.drop(columns = cols_simp, axis=1, inplace=True)

C:\Users\Colin\anaconda3\envs\learn-env\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [12]:
X_train_imp = pd.concat((X_train, df_simp_tranformed), axis=1)
X_train_imp

,Location,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,Month,WindGustDir,WindDir9am,WindDir3pm,RainToday
2867,Albury,16.2,33.0,0.0,26.0,13.0,9.0,53.0,29.0,1018.4,1015.4,NaN,NaN,23.9,31.2,1,NNW,SE,NE,No
110045,Albany,14.3,22.0,0.0,NaN,6.0,13.0,71.0,61.0,1025.4,1021.9,7.0,2.0,17.5,20.5,3,W,SW,SE,No
55289,Ballarat,2.0,9.9,0.0,33.0,11.0,13.0,91.0,94.0,1013.0,1009.2,7.0,8.0,6.6,8.1,6,N,NE,NNE,No
39713,Williamtown,5.4,17.7,0.0,48.0,15.0,28.0,63.0,43.0,1015.6,1012.0,6.0,NaN,11.0,17.3,6,W,WNW,WNW,No
114443,Witchcliffe,12.5,23.4,0.2,43.0,20.0,26.0,59.0,48.0,1017.8,1017.1,NaN,NaN,19.0,22.5,12,S,ESE,SSE,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139707,Darwin,25.4,33.1,0.0,33.0,4.0,22.0,78.0,63.0,1010.2,1006.9,7.0,5.0,29.9,32.7,2,WNW,WSW,WNW,No
121459,Perth,18.2,25.0,0.0,33.0,9.0,9.0,41.0,34.0,1011.1,1009.6,7.0,7.0,19.2,24.4,9,SE,SE,SE,No
940,Albury,-1.4,14.7,0.0,17.0,2.0,9.0,92.0,52.0,1028.9,1024.7,NaN,NaN,3.6,14.1,7,E,NW,ESE,No
18212,NorahHead,14.3,20.8,0.0,44.0,19.0,31.0,70.0,75.0,1026.5,1024.6,NaN,NaN,18.6,19.5,4,S,SW,SSE,No


In [13]:
cols_encode = ['Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday', 'Month']
ohe = OneHotEncoder(sparse = False, handle_unknown='ignore')
ohe_fit = ohe.fit_transform(X_train_imp[cols_encode])
ohe_df = pd.DataFrame(ohe_fit, columns = ohe.get_feature_names(cols_encode), index = X_train_imp.index)
ohe_df

,Location_Adelaide,Location_Albany,Location_Albury,Location_AliceSprings,Location_BadgerysCreek,Location_Ballarat,Location_Bendigo,Location_Brisbane,Location_Cairns,Location_Canberra,...,Month_3,Month_4,Month_5,Month_6,Month_7,Month_8,Month_9,Month_10,Month_11,Month_12
2867,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
110045,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
55289,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
39713,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
114443,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139707,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
121459,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
940,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
18212,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
X_train_int = X_train_imp.drop(cols_encode, axis=1)

In [15]:
X_train_ohe = pd.concat((X_train_int, ohe_df), axis=1)
X_train_ohe

,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,...,Month_3,Month_4,Month_5,Month_6,Month_7,Month_8,Month_9,Month_10,Month_11,Month_12
2867,16.2,33.0,0.0,26.0,13.0,9.0,53.0,29.0,1018.4,1015.4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
110045,14.3,22.0,0.0,NaN,6.0,13.0,71.0,61.0,1025.4,1021.9,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
55289,2.0,9.9,0.0,33.0,11.0,13.0,91.0,94.0,1013.0,1009.2,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
39713,5.4,17.7,0.0,48.0,15.0,28.0,63.0,43.0,1015.6,1012.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
114443,12.5,23.4,0.2,43.0,20.0,26.0,59.0,48.0,1017.8,1017.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139707,25.4,33.1,0.0,33.0,4.0,22.0,78.0,63.0,1010.2,1006.9,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
121459,18.2,25.0,0.0,33.0,9.0,9.0,41.0,34.0,1011.1,1009.6,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
940,-1.4,14.7,0.0,17.0,2.0,9.0,92.0,52.0,1028.9,1024.7,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
18212,14.3,20.8,0.0,44.0,19.0,31.0,70.0,75.0,1026.5,1024.6,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
imputer = KNNImputer(n_neighbors=5, weights="uniform")
imputed = imputer.fit_transform(X_train_ohe)
X_train_KNNI = pd.DataFrame(imputed, index = X_train_ohe.index, columns = X_train_ohe.columns)



In [ ]:
KNN = KNeighborsClassifier()

KNN.fit(X_train_KNNI, y_train)